In [4]:
from tensorflow.python.client import device_lib

device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 16566326356323166718, name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 31873286144
 locality {
   bus_id: 1
   links {
     link {
       device_id: 1
       type: "StreamExecutor"
       strength: 1
     }
   }
 }
 incarnation: 17678996126836637949
 physical_device_desc: "device: 0, name: Tesla V100-PCIE-32GB, pci bus id: 0000:3b:00.0, compute capability: 7.0", name: "/device:GPU:1"
 device_type: "GPU"
 memory_limit: 30456827085
 locality {
   bus_id: 2
   numa_node: 1
   links {
     link {
       type: "StreamExecutor"
       strength: 1
     }
   }
 }
 incarnation: 12571853928925374870
 physical_device_desc: "device: 1, name: Tesla V100-PCIE-32GB, pci bus id: 0000:d8:00.0, compute capability: 7.0"]

In [6]:
import tensorflow as tf; print(tf.__version__)

1.10.1


In [7]:
import keras
import tensorflow as tf

import os
#os.environ["CUDA_VISIBLE_DEVICES"]="0"

import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K

with tf.device('/device:GPU:1'):

    config = tf.ConfigProto()
    config.gpu_options.allow_growth=True
    sess = tf.Session(config=config)

    #config = tf.ConfigProto( device_count = {'GPU': 0 , 'CPU': 30} )
    config = tf.ConfigProto( device_count = {'GPU': 1} )

    keras.backend.set_session(sess)
    
    batch_size = 128
    num_classes = 10
    epochs = 12

    # input image dimensions
    img_rows, img_cols = 28, 28

    # the data, split between train and test sets
    (x_train, y_train), (x_test, y_test) = mnist.load_data(path='./mnist.npz')


    
    if K.image_data_format() == 'channels_first':
        x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
        x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
        input_shape = (1, img_rows, img_cols)
    else:
        x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
        x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
        input_shape = (img_rows, img_cols, 1)

    x_train = x_train.astype('float32')
    x_test = x_test.astype('float32')
    x_train /= 255
    x_test /= 255
    print('x_train shape:', x_train.shape)
    print(x_train.shape[0], 'train samples')
    print(x_test.shape[0], 'test samples')

    # convert class vectors to binary class matrices
    y_train = keras.utils.to_categorical(y_train, num_classes)
    y_test = keras.utils.to_categorical(y_test, num_classes)

    model = Sequential()
    model.add(Conv2D(32, kernel_size=(3, 3),
                     activation='relu',
                     input_shape=input_shape))
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(num_classes, activation='softmax'))

    model.compile(loss=keras.losses.categorical_crossentropy,\
                  optimizer=keras.optimizers.Adadelta(),\
                  metrics=['accuracy'])

    model.fit(x_train, y_train,\
              batch_size=batch_size,\
              epochs=epochs,\
              verbose=1,\
              validation_data=(x_test, y_test))
    score = model.evaluate(x_test, y_test, verbose=0)
    print('Test loss:', score[0])
    print('Test accuracy:', score[1])

x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples
Train on 60000 samples, validate on 10000 samples
Epoch 1/12
60000/60000 [==============================] - 3s 49us/step - loss: 0.2576 - acc: 0.9211 - val_loss: 0.0657 - val_acc: 0.9790
Epoch 2/12
60000/60000 [==============================] - 2s 40us/step - loss: 0.0905 - acc: 0.9732 - val_loss: 0.0396 - val_acc: 0.9864
Epoch 3/12
60000/60000 [==============================] - 3s 42us/step - loss: 0.0662 - acc: 0.9801 - val_loss: 0.0353 - val_acc: 0.9888
Epoch 4/12
60000/60000 [==============================] - 2s 41us/step - loss: 0.0547 - acc: 0.9832 - val_loss: 0.0292 - val_acc: 0.9903
Epoch 5/12
60000/60000 [==============================] - 2s 40us/step - loss: 0.0468 - acc: 0.9855 - val_loss: 0.0292 - val_acc: 0.9909
Epoch 6/12
60000/60000 [==============================] - 2s 41us/step - loss: 0.0401 - acc: 0.9876 - val_loss: 0.0343 - val_acc: 0.9887
Epoch 7/12
60000/60000 [==============================]

In [4]:
del model
from keras import backend as K
K.clear_session()


In [5]:
import gc
gc.collect()

5771

In [3]:
!nvidia-smi

Tue Sep 11 12:04:23 2018       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 390.48                 Driver Version: 390.48                    |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla V100-PCIE...  Off  | 00000000:3B:00.0 Off |                    0 |
| N/A   38C    P0    53W / 250W |  30985MiB / 32510MiB |     70%      Default |
+-------------------------------+----------------------+----------------------+
|   1  Tesla V100-PCIE...  Off  | 00000000:D8:00.0 Off |                    0 |
| N/A   38C    P0    50W / 250W |  31079MiB / 32510MiB |     78%      Default |
+-------------------------------+----------------------+----------------------+
        